# Assignment 2

## Instructions
- Your submission should be the `.ipynb` file with your name,
  like `YusufMesbah.ipynb`. it should include the answers to the questions in
  markdown cells.
- You are expected to follow the best practices for code writing and model
training. Poor coding style will be penalized.
- You are allowed to discuss ideas with your peers, but no sharing of code.
Plagiarism in the code will result in failing. If you use code from the
internet, cite it.
- If the instructions seem vague, use common sense.

# Task 1: ANN (30%)
For this task, you are required to build a fully connect feed-forward ANN model
for a multi-label regression problem.

For the given data, you need do proper data preprocessing, design the ANN model,
then fine-tune your model architecture (number of layers, number of neurons,
activation function, learning rate, momentum, regularization).

For evaluating your model, do $80/20$ train test split.

### Data
You will be working with the data in `Task 1.csv` for predicting students'
scores in 3 different exams: math, reading and writing. The columns include:
 - gender
 - race
 - parental level of education
 - lunch meal plan at school
 - whether the student undertook the test preparation course

In [351]:
!pip3 install -U scikit-image
!pip3 install torch torchvision
!pip3 install -U albumentations
!pip3 install tqdm

In [352]:


import pandas as pd
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.utils.data import DataLoader, Dataset, Subset

In [353]:
# TODO: Implement task 1

df = pd.read_csv('Task 1.csv')

df = df.drop(['lunch'], axis=1)

df.head(5)

,gender,race/ethnicity,parental level of education,test preparation course,math score,reading score,writing score
0,male,group A,high school,completed,67,67,63
1,female,group D,some high school,none,40,59,55
2,male,group E,some college,none,59,60,50
3,male,group B,high school,none,77,78,68
4,male,group E,associate's degree,completed,78,73,68


In [354]:
assert not df.isnull().values.any()

df['gender'] = df['gender'].astype('category')
df['gender'] = df['gender'].cat.codes

df['test preparation course'] = df['test preparation course'].astype('category')
df['test preparation course'] = df['test preparation course'].cat.codes

df['race/ethnicity'] = df['race/ethnicity'].astype('category')
df['race/ethnicity'] = df['race/ethnicity'].cat.codes

df['parental level of education'] = df['parental level of education'].astype('category')
df['parental level of education'] = df['parental level of education'].cat.codes

scaler = MinMaxScaler()
scaler.fit(df)
df = scaler.transform(df)

df = pd.DataFrame(df)

df.head()

,0,1,2,3,4,5,6
0,1.0,0.00,0.4,0.0,0.620690,0.547945,0.519481
1,0.0,0.75,1.0,1.0,0.310345,0.438356,0.415584
2,1.0,1.00,0.8,1.0,0.528736,0.452055,0.350649
3,1.0,0.25,0.4,1.0,0.735632,0.698630,0.584416
4,1.0,1.00,0.0,0.0,0.747126,0.630137,0.584416


In [355]:
class MyDataset(Dataset):

  def __init__(self, file_name):


    x=file_name.iloc[:,0:4].values
    y=file_name.iloc[:,4:7].values

    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.float32)


  def __len__(self):
    return len(self.y_train)

  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]



In [356]:
dataset = MyDataset(df)
# for i, (data, labels) in enumerate(dataset):
#     print(data.shape, labels.shape)
#     print(data,labels)
#     break

In [417]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle = False)

In [418]:
import torch as T
class Net(T.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.hid1 = T.nn.Linear(4, 10)  # 6-(10-10)-3
    self.hid2 = T.nn.Linear(10, 10)
    # self.hid3 = T.nn.Linear(20, 40)
    # self.hid4 = T.nn.Linear(40, 40)
    # self.hid5 = T.nn.Linear(40, 10)
    self.oupt = T.nn.Linear(10, 3)

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    # T.nn.init.xavier_uniform_(self.hid3.weight)
    # T.nn.init.zeros_(self.hid3.bias)
    # T.nn.init.xavier_uniform_(self.hid4.weight)
    # T.nn.init.zeros_(self.hid4.bias)
    # T.nn.init.xavier_uniform_(self.hid5.weight)
    # T.nn.init.zeros_(self.hid5.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight) 
    T.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = T.tanh(self.hid1(x))
    z = T.tanh(self.hid2(z))
    # z = T.tanh(self.hid3(z))
    # z = (self.hid4(z))
    # z = T.tanh(self.hid5(z))
    z = T.sigmoid(self.oupt(z))  # assumes CrossEntropyLoss()
    return z




In [419]:
from torch import optim, nn
net = Net()
optimizer = torch.optim.SGD(net.parameters(),lr = 0.02)
mse = nn.MSELoss()
epoch = 10
loss_list = []
accuracy_list = []
iteration_list = []


In [420]:
from torch.autograd import Variable
from tqdm import tqdm

In [423]:
n_epochs = 1000
count = 0
for epoch in range(n_epochs):
    for features, labels in tqdm(train_loader):
        features, labels = Variable(features), Variable(labels)

        # zero out gradients from previous iteration
        optimizer.zero_grad()

        # forward propagation
        output = net(features)
        
        # calculate loss
        loss = mse(output, labels)

        # backprop
        loss.backward()
        
        # update params (gradient descent)
        optimizer.step()
            

for features, labels in train_loader:
    with torch.no_grad():
        features, labels = Variable(features), Variable(labels)
        output = net(features)
        print(features[0], output[0])
    break

100%|██████████| 10/10 [00:00<00:00, 1431.45it/s]


tensor([1.0000, 0.0000, 0.4000, 0.0000]) tensor([0.6238, 0.5667, 0.5814])


### Questions
1. What preprocessing techniques did you use? Why?
    - *Answer*
2. Describe the fine-tuning process and how you reached your model architecture.
    - *Answer*

# Task 2: CNN (40%)
For this task, you will be doing image classification:
- First, adapt your best model from Task 1 to work on this task, and
fit it on the new data. Then, evaluate its performance.
- After that, build a CNN model for image classification.
- Compare both models in terms of accuracy, number of parameters and speed of
inference (the time the model takes to predict 50 samples).

For the given data, you need to do proper data preprocessing and augmentation,
data loaders.
Then fine-tune your model architecture (number of layers, number of filters,
activation function, learning rate, momentum, regularization).

### Data
You will be working with the data in `triple_mnist.zip` for predicting 3-digit
numbers writen in the image. Each image contains 3 digits similar to the
following example (whose label is `039`):

![example](https://github.com/shaohua0116/MultiDigitMNIST/blob/master/asset/examples/039/0_039.png?raw=true)

In [364]:
# TODO: Implement task 2

### Questions
1. What preprocessing techniques did you use? Why?
    - *Answer*
2. What data augmentation techniques did you use?
    - *Answer*
3. Describe the fine-tuning process and how you reached your final CNN model.
    - *Answer*

# Task 3: Decision Trees and Ensemble Learning (15%)

For the `loan_data.csv` data, predict if the bank should give a loan or not.
You need to do the following:
- Fine-tune a decision tree on the data
- Fine-tune a random forest on the data
- Compare their performance
- Visualize your DT and one of the trees from the RF

For evaluating your models, do $80/20$ train test split.

### Data
- `credit.policy`: Whether the customer meets the credit underwriting criteria.
- `purpose`: The purpose of the loan.
- `int.rate`: The interest rate of the loan.
- `installment`: The monthly installments owed by the borrower if the loan is funded.
- `log.annual.inc`: The natural logarithm of the self-reported annual income of the borrower.
- `dti`: The debt-to-income ratio of the borrower.
- `fico`: The FICO credit score of the borrower.
- `days.with.cr.line`: The number of days the borrower has had a credit line.
- `revol.bal`: The borrower's revolving balance.
- `revol.util`: The borrower's revolving line utilization rate.

In [365]:
# TODO: Implement task 3

### Questions
1. How did the DT compare to the RF in performance? Why?
    - *Answer*
2. After fine-tuning, how does the max depth in DT compare to RF? Why?
    - *Answer*
3. What is ensemble learning? What are its pros and cons?
    - *Answer*
4. Briefly explain 2 types of boosting methods and 2 types of bagging methods.
Which of these categories does RF fall under?
    - *Answer*

# Task 4: Domain Gap (15%)

Evaluate your CNN model from task 2 on SVHN data without retraining your model.

In [366]:
# TODO: Implement task 4

### Questions
1. How did your model perform? Why is it better/worse?
    - *Answer*
2. What is domain gap in the context of ML?
    - *Answer*
3. Suggest two ways through which the problem of domain gap can be tackled.
    - *Answer*